---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [6]:
def date_sorter():
    
    import pandas as pd

    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)

    
    import re

    ## LISTA DE PATTERNS ORDENADA SEGÚN INSTRUCCIONES
    pattern1 = r'(\d{1,2}[\/-]\d{1,2}[\/-]\d{2,4})' 
    pattern2 = r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,.]?\W{1,2}\d{1,2}\W{1,2}[12]\d{3})' #Mejor señalando abreviación de meses pq sino se puede filtrar otra string con mayus de comienzo y minúscula siguiente
    pattern3 = r'(\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\W?\s[12]\d{3})'  
    pattern4 = r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\W?\s\d{1,2}[st|nd|rd|th]+\W\s[12]\d{3})' #ordinales: first, second, third, fourth
    pattern5 = r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s[12]\d{3})'  #primera letra señalando mes es en mayúscula
    pattern6 = r'(\d{1,2}\/\d{2,4})' #si año se señala en 4 cifras entonces el año es o los 1000 o 2000, i.e. parte con 1 o 2
    pattern7 = r'([12]\d{3})'

    patterns = [pattern1, pattern2, pattern3, pattern4, pattern5, pattern6, pattern7]
    reverse_patterns = patterns

    months = {'jan': '01', 'feb': '02', 'mar': '03', 'apr': '04', 'may':'05',
              'jun':'06', 'jul': '07', 'aug': '08', 'sep': '09', 'oct': '10', 'nov': '11', 'dec': '12'}

    data = df.str.extract('(' + '|'.join(reverse_patterns) + ')') # dentro del extract (X) para que sea 1 grupo nada más, 1 columna
    data = data.rename(columns={0: 'dates'})
    data = data['dates']

    ## IDENTIFICAR SI HAY NaN EN DATA LUEGO DE EXTRACCIÓN POR PATTERNS
    # null_values = pd.isnull(data).nonzero()[0]
    # print(len(null_values))

    data = data.apply(lambda x: re.sub(r'(\d{1,2})[\/-](\d{1,2})[\/-](\d{1,2})', r'\2/\1/\3', x)) #CAMBIAR ORDEN DE XX/XX/XX EN MM/DD/YY POR DD/MM/YY (Assume all dates in xx/xx/xx format are mm/dd/yy)
    
    data = data.str.replace(r'(^\d{1}[\/-])', lambda x: '0'+x.groups()[0]) #AGREGAR 0 EN DÍAS CUANDO ESTA EN SINGULAR PARA TENER FORMATO DD
    data = data.str.replace(r'(?<=[\/|-])(\d{1})?(?=[\/|-])', lambda x: '0'+x.groups()[0]) #AGREGAR 0 PARA COMPLETAR PRIMEROS 9 MESES CON FORMATO MM
    data = data.str.replace(r'(?<=[\/|-])(\d{2}$)', lambda x: '19'+x.groups()[0]) # YY to 19YY
    data = data.str.replace(r'([A-Z]\w+)', lambda x: x.groups()[0][:3].lower())  #April to Apr
    data = data.str.replace(r'(^(\d{2}|\w{3})[\/\s-]\d{4}$)', lambda x: '01/'+x.groups()[0]) #If the day is missing (e.g. aug 2009), assume it is the first day of the month (e.g. aug 1, 2009).
    data = data.str.replace(r'(^\d{2}[\/-]\d{4}$)', lambda x: '01'+x.groups()[0]) #If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
    data = data.str.replace(r'(^\d{4}$)', lambda x: '01/01/'+x.groups()[0]) #If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
    data = data.str.replace(' ', '/').str.replace('-', '/').str.replace('.', '').str.replace(',', '')
    
    data = data.apply(lambda x: re.sub(r'([a-z]{3})[\/](\d{2})[\/](\d{4})', r'\2/\1/\3', x)) #CAMBIAR ORDEN DE MES EN 3 LETRAS EN PRIMERA COLUMNA AL MES EN SEGUNDA COLUMNA

    

    data = data.str.extractall('(\d{2})[\/](.*)[\/](\d{4})').loc[:, [0, 1, 2]]
    index = data.index.droplevel(1)
    data = data.reset_index(drop=True)

    data.columns = ['dd','mm','yyyy']
    data.index = index
    data['mm'].replace(months, inplace=True)
    data['date'] = data['mm'] + '/' + data['dd'] + '/' + data['yyyy']  #gran diferencia en este orden
    
    
    ## TO_DATETIME INPUT-OUTPUT:
    #import pandas as pd
    # # input in mm.dd.yyyy format
    # date = ['01.02.2019']
    # # output in yyyy-mm-dd format
    # print(pd.to_datetime(date))
    
    
    data.drop(['dd','mm','yyyy'], axis=1, inplace=True)   

    data = data.apply(pd.to_datetime).sort_values('date')
    s = pd.Series(data.index)
    
    return s

